# Mundo de los bloques

El mundo de los bloques es un ejemplo clásico en el campo de la inteligencia artificial y la planificación automatizada. Consiste en un conjunto de bloques que pueden estar apilados unos sobre otros o situados sobre una mesa. Cada bloque tiene un nombre único, y las acciones que podemos realizar son limitadas y específicas: recoger un bloque, poner un bloque sobre otro bloque, apilar un bloque sobre la mesa, o desapilar un bloque de otro bloque.

**Objetivo**

El objetivo en el mundo de los bloques es llegar desde un estado inicial a un estado objetivo predeterminado. Los estados están definidos por la posición de los bloques y las relaciones entre ellos (si están en la mesa, si un bloque está sobre otro, si un bloque está claro o no, etc.). Para alcanzar el estado objetivo, disponemos de una serie de acciones que nos permiten manipular los bloques de manera controlada.

**Acciones**

Las acciones básicas en el mundo de los bloques son:

- Recoger (Pick Up): Permite recoger un bloque de la mesa si el bloque está claro y la mano está vacía.
- Poner (Put Down): Permite colocar un bloque en la mesa si se está sosteniendo el bloque.
- Apilar (Stack): Permite apilar un bloque sobre otro si se está sosteniendo el bloque y el bloque destino está claro.
- Desapilar (Unstack): Permite desapilar un bloque de otro si el bloque superior está claro y la mano está vacía.

Cada acción tiene precondiciones (condiciones que deben cumplirse para que la acción sea posible), una lista de efectos de borrado (proposiciones que dejan de ser verdaderas después de la acción) y una lista de efectos de adición (proposiciones que se vuelven verdaderas después de la acción).

A continuación, mostramos la planificación para llegar del estado inicial, que se muestra en la imagen, al estado objetivo.

<center>
<img src="../images/blocks.png" width="700" height="630">
</center>

Importamos las librerías necesarias

In [9]:
import sys
import os

sys.path.append("../")
from GraphPlanning import Proposition, Action, Node, fastforward_A_star

Definimos las acciones, donde para cada acción tendremos un nombre, una lista de precondiciones, una lista de literales por borrar y una lista de efectos (literales que se agregan)

In [10]:
pickUpA = Action(
    'Pick Up A', 
    [Proposition('OnTable', 'A'), Proposition('Clear', 'A'), Proposition('HandEmpty')],            
    [Proposition('OnTable', 'A'), Proposition('Clear', 'A'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'A')],
)

pickUpB = Action(
    'Pick Up B', 
    [Proposition('OnTable', 'B'), Proposition('Clear', 'B'), Proposition('HandEmpty')],            
    [Proposition('OnTable', 'B'), Proposition('Clear', 'B'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'B')],
)

pickUpC = Action(
    'Pick Up C', 
    [Proposition('OnTable', 'C'), Proposition('Clear', 'C'), Proposition('HandEmpty')],            
    [Proposition('OnTable', 'C'), Proposition('Clear', 'C'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'C')],
)

putDownA = Action(
    'Put down A', 
    [Proposition('Holding', 'A')],            
    [Proposition('Holding', 'A')], 
    [Proposition('OnTable', 'A'), Proposition('Clear', 'A'), Proposition('HandEmpty')],
)

putDownB = Action(
    'Put down B', 
    [Proposition('Holding', 'B')],            
    [Proposition('Holding', 'B')], 
    [Proposition('OnTable', 'B'), Proposition('Clear', 'B'), Proposition('HandEmpty')],
)

putDownC = Action(
    'Put down C', 
    [Proposition('Holding', 'C')],            
    [Proposition('Holding', 'C')], 
    [Proposition('OnTable', 'C'), Proposition('Clear', 'C'), Proposition('HandEmpty')],
)

stackAB = Action(
    'Stack A on B', 
    [Proposition('Holding', 'A'), Proposition('Clear', 'B')],            
    [Proposition('Holding', 'A'), Proposition('Clear', 'B')], 
    [Proposition('On', 'A,B'), Proposition('Clear', 'A'), Proposition('HandEmpty')],
)

stackBA = Action(
    'Stack B on A', 
    [Proposition('Holding', 'B'), Proposition('Clear', 'A')],            
    [Proposition('Holding', 'B'), Proposition('Clear', 'A')], 
    [Proposition('On', 'B,A'), Proposition('Clear', 'B'), Proposition('HandEmpty')],
)

stackAC = Action(
    'Stack A on C', 
    [Proposition('Holding', 'A'), Proposition('Clear', 'C')],            
    [Proposition('Holding', 'A'), Proposition('Clear', 'C')], 
    [Proposition('On', 'A,C'), Proposition('Clear', 'A'), Proposition('HandEmpty')],
)

stackCA = Action(
    'Stack C on A', 
    [Proposition('Holding', 'C'), Proposition('Clear', 'A')],            
    [Proposition('Holding', 'C'), Proposition('Clear', 'A')], 
    [Proposition('On', 'C,A'), Proposition('Clear', 'C'), Proposition('HandEmpty')],
)

stackBC = Action(
    'Stack B on C', 
    [Proposition('Holding', 'B'), Proposition('Clear', 'C')],            
    [Proposition('Holding', 'B'), Proposition('Clear', 'C')], 
    [Proposition('On', 'B,C'), Proposition('Clear', 'B'), Proposition('HandEmpty')],
)

stackCB = Action(
    'Stack C on B', 
    [Proposition('Holding', 'C'), Proposition('Clear', 'B')],            
    [Proposition('Holding', 'C'), Proposition('Clear', 'B')], 
    [Proposition('On', 'C,B'), Proposition('Clear', 'C'), Proposition('HandEmpty')],
)

unstackAB = Action(
    'Unstack A from B', 
    [Proposition('On', 'A,B'), Proposition('Clear', 'A'), Proposition('HandEmpty')],            
    [Proposition('On', 'A,B'), Proposition('Clear', 'A'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'A'), Proposition('Clear', 'B')],
)

unstackBA = Action(
    'Unstack B from A', 
    [Proposition('On', 'B,A'), Proposition('Clear', 'B'), Proposition('HandEmpty')],            
    [Proposition('On', 'B,A'), Proposition('Clear', 'B'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'B'), Proposition('Clear', 'A')],
)

unstackAC = Action(
    'Unstack A from C', 
    [Proposition('On', 'A,C'), Proposition('Clear', 'A'), Proposition('HandEmpty')],            
    [Proposition('On', 'A,C'), Proposition('Clear', 'A'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'A'), Proposition('Clear', 'C')],
)

unstackCA = Action(
    'Unstack C from A', 
    [Proposition('On', 'C,A'), Proposition('Clear', 'C'), Proposition('HandEmpty')],            
    [Proposition('On', 'C,A'), Proposition('Clear', 'C'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'C'), Proposition('Clear', 'A')],
)

unstackBC = Action(
    'Unstack B from C', 
    [Proposition('On', 'B,C'), Proposition('Clear', 'B'), Proposition('HandEmpty')],            
    [Proposition('On', 'B,C'), Proposition('Clear', 'B'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'B'), Proposition('Clear', 'C')],
)

unstackCB = Action(
    'Unstack C from B', 
    [Proposition('On', 'C,B'), Proposition('Clear', 'C'), Proposition('HandEmpty')],            
    [Proposition('On', 'C,B'), Proposition('Clear', 'C'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'C'), Proposition('Clear', 'B')],
)


Ahora definimos el estado incial y objetivo

In [11]:
initial_state = [Proposition('On', 'C,A'), Proposition('OnTable', 'A'), Proposition('OnTable', 'B'), Proposition('Clear', 'C'), Proposition('Clear', 'B'), Proposition('HandEmpty')]
goal = [Proposition('On', 'A,B'), Proposition('On','B,C')]

Realizamos la búsqueda del plan con el algoritmo A* y la heurística del Fast Forward Planner

In [12]:
node = Node(initial_state, 0, float('inf'), [])

In [13]:
actions = [
    pickUpA,
    pickUpB,
    pickUpC,
    putDownA,
    putDownB,
    putDownC,
    stackAB,
    stackBA,
    stackAC,
    stackCA,
    stackBC,
    stackCB,
    unstackAB,
    unstackBA,
    unstackAC,
    unstackCA,
    unstackBC,
    unstackCB
]

In [14]:
plan = fastforward_A_star(node, goal, actions)

In [15]:
plan = ' -> '.join([str(x) for x in plan])

In [16]:
plan

'Unstack C from A -> Put down C -> Pick Up B -> Stack B on C -> Pick Up A -> Stack A on B'

Finalmente obtenemos el plan el cual consiste en:
1. Desapliar C de A
2. Poner C en la mesa
3. Recoger B
4. Apilar B sobre C
5. Recoger A
6. Apilar A sobre B
